In [1]:
# DataFrame
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import spacy

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
from collections import Counter
import time

In [2]:
#Importing Datasets
song_data = pd.read_csv('genres_v2.csv')
lyric_data = pd.read_csv('lyrics-data.csv')
id_data = pd.read_csv('songs.csv')
song_data

/Users/pradyumn/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.059800,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.401000,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.013800,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.023700,0.000000,0.1140,0.1750,...,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.217000,0.000000,0.1660,0.5910,...,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42300,0.528,0.693,4,-5.148,1,0.0304,0.031500,0.000345,0.1210,0.3940,...,46bXU7Sgj7104ZoXxzz9tM,spotify:track:46bXU7Sgj7104ZoXxzz9tM,https://api.spotify.com/v1/tracks/46bXU7Sgj710...,https://api.spotify.com/v1/audio-analysis/46bX...,269208,4,hardstyle,NaN,20995.0,Euphoric Hardstyle
42301,0.517,0.768,0,-7.922,0,0.0479,0.022500,0.000018,0.2050,0.3830,...,0he2ViGMUO3ajKTxLOfWVT,spotify:track:0he2ViGMUO3ajKTxLOfWVT,https://api.spotify.com/v1/tracks/0he2ViGMUO3a...,https://api.spotify.com/v1/audio-analysis/0he2...,210112,4,hardstyle,NaN,20996.0,Greatest Hardstyle Playlist
42302,0.361,0.821,8,-3.102,1,0.0505,0.026000,0.000242,0.3850,0.1240,...,72DAt9Lbpy9EUS29OzQLob,spotify:track:72DAt9Lbpy9EUS29OzQLob,https://api.spotify.com/v1/tracks/72DAt9Lbpy9E...,https://api.spotify.com/v1/audio-analysis/72DA...,234823,4,hardstyle,NaN,20997.0,Best of Hardstyle 2020
42303,0.477,0.921,6,-4.777,0,0.0392,0.000551,0.029600,0.0575,0.4880,...,6HXgExFVuE1c3cq9QjFCcU,spotify:track:6HXgExFVuE1c3cq9QjFCcU,https://api.spotify.com/v1/tracks/6HXgExFVuE1c...,https://api.spotify.com/v1/audio-analysis/6HXg...,323200,4,hardstyle,NaN,20998.0,Euphoric Hardstyle


In [3]:
lyric_data

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [4]:
id_data

,year,songtitle,artistname,songID,artistID,timesignature,timesignature_confidence,loudness,tempo,tempo_confidence,...,timbre_7_max,timbre_8_min,timbre_8_max,timbre_9_min,timbre_9_max,timbre_10_min,timbre_10_max,timbre_11_min,timbre_11_max,Top10
0,2010,This Is the House That Doubt Built,A Day to Remember,SOBGGAB12C5664F054,AROBSHL1187B9AFB01,3,0.853,-4.262,91.525,0.953,...,82.475,-52.025,39.116,-35.368,71.642,-126.440,18.658,-44.770,25.989,0
1,2010,Sticks & Bricks,A Day to Remember,SOPAQHU1315CD47F31,AROBSHL1187B9AFB01,4,1.000,-4.051,140.048,0.921,...,106.918,-61.320,35.378,-81.928,74.574,-103.808,121.935,-38.892,22.513,0
2,2010,All I Want,A Day to Remember,SOOIZOU1376E7C6386,AROBSHL1187B9AFB01,4,1.000,-3.571,160.512,0.489,...,80.621,-59.773,45.979,-46.293,59.904,-108.313,33.300,-43.733,25.744,0
3,2010,It's Complicated,A Day to Remember,SODRYWD1315CD49DBE,AROBSHL1187B9AFB01,4,1.000,-3.815,97.525,0.794,...,96.675,-78.660,41.088,-49.194,95.440,-102.676,46.422,-59.439,37.082,0
4,2010,2nd Sucks,A Day to Remember,SOICMQB1315CD46EE3,AROBSHL1187B9AFB01,4,0.788,-4.707,140.053,0.286,...,110.332,-56.450,37.555,-48.588,67.570,-52.796,22.888,-50.414,32.758,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7569,1990,She's Gone,Y&T,SOSIEQB12A6D4FC59D,ARGQANQ11F50C4769E,4,1.000,-10.197,93.140,0.859,...,38.120,-66.638,50.105,-40.692,43.763,-59.707,49.414,-53.970,68.303,0
7570,1990,Let It Out,Y&T,SOIGIQI12A6D4FC59E,ARGQANQ11F50C4769E,4,1.000,-12.392,79.858,0.196,...,83.454,-83.707,44.124,-51.628,69.929,-97.153,36.745,-61.243,56.902,0
7571,1990,Ten Lovers,Y&T,SOLNEQO12A6D4FC59F,ARGQANQ11F50C4769E,4,0.984,-10.304,91.760,0.592,...,103.413,-101.464,36.152,-45.387,48.352,-57.103,67.641,-53.729,65.176,0
7572,1990,Goin' Off The Deep End,Y&T,SOEMJEP12A58A7E7B5,ARGQANQ11F50C4769E,4,0.907,-9.295,110.907,0.838,...,108.688,-55.893,42.222,-76.631,68.336,-83.284,56.476,-51.687,59.427,0


In [5]:
song_data.info()
print('\nColumns in song_data:',song_data.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      42305 non-null  float64
 1   energy            42305 non-null  float64
 2   key               42305 non-null  int64  
 3   loudness          42305 non-null  float64
 4   mode              42305 non-null  int64  
 5   speechiness       42305 non-null  float64
 6   acousticness      42305 non-null  float64
 7   instrumentalness  42305 non-null  float64
 8   liveness          42305 non-null  float64
 9   valence           42305 non-null  float64
 10  tempo             42305 non-null  float64
 11  type              42305 non-null  object 
 12  id                42305 non-null  object 
 13  uri               42305 non-null  object 
 14  track_href        42305 non-null  object 
 15  analysis_url      42305 non-null  object 
 16  duration_ms       42305 non-null  int64 

In [6]:
lyric_data.info()
print('\nColumns in lyric_data:',lyric_data.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379931 entries, 0 to 379930
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ALink     379930 non-null  object
 1   SName     379928 non-null  object
 2   SLink     379930 non-null  object
 3   Lyric     379854 non-null  object
 4   language  365296 non-null  object
dtypes: object(5)
memory usage: 14.5+ MB

Columns in lyric_data: Index(['ALink', 'SName', 'SLink', 'Lyric', 'language'], dtype='object')


In [7]:
id_data.info()
print('\nColumns in id_data:',id_data.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7574 entries, 0 to 7573
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      7574 non-null   int64  
 1   songtitle                 7574 non-null   object 
 2   artistname                7574 non-null   object 
 3   songID                    7574 non-null   object 
 4   artistID                  7574 non-null   object 
 5   timesignature             7574 non-null   int64  
 6   timesignature_confidence  7574 non-null   float64
 7   loudness                  7574 non-null   float64
 8   tempo                     7574 non-null   float64
 9   tempo_confidence          7574 non-null   float64
 10  key                       7574 non-null   int64  
 11  key_confidence            7574 non-null   float64
 12  energy                    7574 non-null   float64
 13  pitch                     7574 non-null   float64
 14  timbre_0

In [8]:
# Dropping uneccessary columns 
lyric_data = lyric_data.drop(['SLink'], axis = 1)

song_data = song_data.drop(['type', 'id', 'uri', 'track_href', 'analysis_url',
       'time_signature', 'Unnamed: 0', 'title'], axis = 1)

id_data = id_data.drop(['timesignature_confidence', 'tempo', 'loudness',
       'tempo_confidence', 'key_confidence', 'energy', 'pitch',
       'timbre_0_min', 'timbre_0_max', 'timbre_1_min', 'timbre_1_max',
       'timbre_2_min', 'timbre_2_max', 'timbre_3_min', 'timbre_3_max',
       'timbre_4_min', 'timbre_4_max', 'timbre_5_min', 'timbre_5_max',
       'timbre_6_min', 'timbre_6_max', 'timbre_7_min', 'timbre_7_max',
       'timbre_8_min', 'timbre_8_max', 'timbre_9_min', 'timbre_9_max',
       'timbre_10_min', 'timbre_10_max', 'timbre_11_min', 'timbre_11_max',
       'Top10', 'year', 'key', 'timesignature'], axis = 1)

lyric_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379931 entries, 0 to 379930
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ALink     379930 non-null  object
 1   SName     379928 non-null  object
 2   Lyric     379854 non-null  object
 3   language  365296 non-null  object
dtypes: object(4)
memory usage: 11.6+ MB


In [9]:
song_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      42305 non-null  float64
 1   energy            42305 non-null  float64
 2   key               42305 non-null  int64  
 3   loudness          42305 non-null  float64
 4   mode              42305 non-null  int64  
 5   speechiness       42305 non-null  float64
 6   acousticness      42305 non-null  float64
 7   instrumentalness  42305 non-null  float64
 8   liveness          42305 non-null  float64
 9   valence           42305 non-null  float64
 10  tempo             42305 non-null  float64
 11  duration_ms       42305 non-null  int64  
 12  genre             42305 non-null  object 
 13  song_name         21519 non-null  object 
dtypes: float64(9), int64(3), object(2)
memory usage: 4.5+ MB


In [10]:
id_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7574 entries, 0 to 7573
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   songtitle   7574 non-null   object
 1   artistname  7574 non-null   object
 2   songID      7574 non-null   object
 3   artistID    7574 non-null   object
dtypes: object(4)
memory usage: 236.8+ KB


In [11]:
#Looking for null values in datasets
print('song_data:')
print(song_data.isnull().sum())
print('\n')
print('lyric_data:')
print(lyric_data.isnull().sum())
print('\n')
print('id_data:')
print(id_data.isnull().sum())

song_data:
danceability            0
energy                  0
key                     0
loudness                0
mode                    0
speechiness             0
acousticness            0
instrumentalness        0
liveness                0
valence                 0
tempo                   0
duration_ms             0
genre                   0
song_name           20786
dtype: int64


lyric_data:
ALink           1
SName           3
Lyric          77
language    14635
dtype: int64


id_data:
songtitle     0
artistname    0
songID        0
artistID      0
dtype: int64


In [12]:
#Dropping NULL Values
song_data = song_data.dropna()
print(song_data.count())

danceability        21519
energy              21519
key                 21519
loudness            21519
mode                21519
speechiness         21519
acousticness        21519
instrumentalness    21519
liveness            21519
valence             21519
tempo               21519
duration_ms         21519
genre               21519
song_name           21519
dtype: int64


In [13]:
#Dropping NULL Values
lyric_data = lyric_data.dropna()
print(lyric_data.count())

ALink       365294
SName       365294
Lyric       365294
language    365294
dtype: int64


In [14]:
#Looking for duplicates in each dataset
duplicate_rows_df = song_data[song_data.duplicated()]
print('number of duplicate rows: ', duplicate_rows_df.shape)

number of duplicate rows:  (438, 14)


In [15]:
#Looking for duplicates in each dataset
duplicate_rows_df = lyric_data[lyric_data.duplicated()]
print('number of duplicate rows: ', duplicate_rows_df.shape)

number of duplicate rows:  (3, 4)


In [16]:
#Dropping the duplicates
song_data = song_data.drop_duplicates()
print('song_data after removing duplicates:')
print(song_data.count())

song_data after removing duplicates:
danceability        21081
energy              21081
key                 21081
loudness            21081
mode                21081
speechiness         21081
acousticness        21081
instrumentalness    21081
liveness            21081
valence             21081
tempo               21081
duration_ms         21081
genre               21081
song_name           21081
dtype: int64


In [17]:
#Dropping the duplicates
lyric_data = lyric_data.drop_duplicates()
print('lyric_data after removing duplicates:')
print(lyric_data.count())

lyric_data after removing duplicates:
ALink       365291
SName       365291
Lyric       365291
language    365291
dtype: int64


In [18]:
#Changing artist name in lyric data to match format in song_data
lyric_data['ALink'] = lyric_data['ALink'].str.replace('-',' ')
lyric_data['ALink'] = lyric_data['ALink'].str.replace('/','')
lyric_data

,ALink,SName,Lyric,language
0,ivete sangalo,Arerê,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,ivete sangalo,Se Eu Não Te Amasse Tanto Assim,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,ivete sangalo,Céu da Boca,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,ivete sangalo,Quando A Chuva Passar,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,ivete sangalo,Sorte Grande,A minha sorte grande foi você cair do céu\nMin...,pt
...,...,...,...,...
379926,clegg johnny,The Waiting,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,clegg johnny,Too Early For The Sky,I nearly disappeared into the mouth of a croco...,en
379928,clegg johnny,Warsaw 1943 (I Never Betrayed The Revolution),"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,clegg johnny,When The System Has Fallen,Sweat in the heat for days on end\nwaiting for...,en


In [19]:
#Converting the artist name to lower case for the song_data dataset(to match the other dataset)


temp_id = id_data
temp_id['artistname'] = id_data['artistname'].str.lower()


## Merging the datasets

In [20]:
#Using the Song id as the key for merging the datasets

data = song_data.merge(temp_id, left_on = ['song_name'], right_on = ['songtitle'],how='inner').drop(['songtitle'], axis = 1)
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 2813
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      2814 non-null   float64
 1   energy            2814 non-null   float64
 2   key               2814 non-null   int64  
 3   loudness          2814 non-null   float64
 4   mode              2814 non-null   int64  
 5   speechiness       2814 non-null   float64
 6   acousticness      2814 non-null   float64
 7   instrumentalness  2814 non-null   float64
 8   liveness          2814 non-null   float64
 9   valence           2814 non-null   float64
 10  tempo             2814 non-null   float64
 11  duration_ms       2814 non-null   int64  
 12  genre             2814 non-null   object 
 13  song_name         2814 non-null   object 
 14  artistname        2814 non-null   object 
 15  songID            2814 non-null   object 
 16  artistID          2814 non-null   object 


In [21]:
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre,song_name,artistname,songID,artistID
0,0.849,0.648,7,-6.188,1,0.0832,0.072500,0.005920,0.0984,0.1960,212.150,111020,Dark Trap,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A
1,0.849,0.648,7,-6.188,1,0.0832,0.072500,0.005920,0.0984,0.1960,106.075,111020,Underground Rap,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A
2,0.859,0.483,7,-11.992,1,0.1250,0.046600,0.004190,0.1140,0.2190,106.054,106606,Underground Rap,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A
3,0.849,0.648,7,-6.188,1,0.0832,0.072500,0.005920,0.0984,0.1960,212.150,111020,Trap Metal,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A
4,0.836,0.706,1,-4.065,1,0.0497,0.000052,0.000000,0.0927,0.0374,120.045,76042,Dark Trap,Blood,band of skulls,SOJXSLN1311AFDB3FE,ARONQXK122BCFC9386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2809,0.897,0.466,10,-9.053,0,0.2030,0.187000,0.000001,0.0757,0.8840,200.020,223133,Hiphop,Music,madonna,SOSGYSS12A6701FA4A,ARBEOHF1187B9B044D
2810,0.592,0.691,2,-8.435,1,0.2360,0.078700,0.000001,0.2770,0.5040,171.946,319392,Hiphop,Bring the Ruckus,manafest,SOROEKC1312A8AC135,ARYZ8BG1187FB52ABB
2811,0.707,0.427,5,-12.216,0,0.0698,0.339000,0.000000,0.1050,0.5410,176.160,284693,Hiphop,Hey Lover,ll cool j,SOYWIJP137E054C163,ARNHMFD1187FB3B3F6
2812,0.939,0.675,0,-7.232,1,0.2110,0.091500,0.000000,0.0565,0.7950,193.836,248573,Hiphop,Shoop,salt-n-pepa,SOIXLDB13C1AD1A6D3,ARUQ6301187FB54EBA


In [22]:
data = data.merge(lyric_data, left_on = ['song_name','artistname'], right_on = ['SName','ALink'],how='inner').drop(['SName','ALink'], axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1127 entries, 0 to 1126
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1127 non-null   float64
 1   energy            1127 non-null   float64
 2   key               1127 non-null   int64  
 3   loudness          1127 non-null   float64
 4   mode              1127 non-null   int64  
 5   speechiness       1127 non-null   float64
 6   acousticness      1127 non-null   float64
 7   instrumentalness  1127 non-null   float64
 8   liveness          1127 non-null   float64
 9   valence           1127 non-null   float64
 10  tempo             1127 non-null   float64
 11  duration_ms       1127 non-null   int64  
 12  genre             1127 non-null   object 
 13  song_name         1127 non-null   object 
 14  artistname        1127 non-null   object 
 15  songID            1127 non-null   object 
 16  artistID          1127 non-null   object 


In [23]:
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre,song_name,artistname,songID,artistID,Lyric,language
0,0.849,0.648,7,-6.188,1,0.0832,0.072500,0.005920,0.0984,0.1960,212.150,111020,Dark Trap,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A,"You've come so far, well done darling\nWe knew...",en
1,0.849,0.648,7,-6.188,1,0.0832,0.072500,0.005920,0.0984,0.1960,106.075,111020,Underground Rap,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A,"You've come so far, well done darling\nWe knew...",en
2,0.859,0.483,7,-11.992,1,0.1250,0.046600,0.004190,0.1140,0.2190,106.054,106606,Underground Rap,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A,"You've come so far, well done darling\nWe knew...",en
3,0.849,0.648,7,-6.188,1,0.0832,0.072500,0.005920,0.0984,0.1960,212.150,111020,Trap Metal,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A,"You've come so far, well done darling\nWe knew...",en
4,0.836,0.706,1,-4.065,1,0.0497,0.000052,0.000000,0.0927,0.0374,120.045,76042,Dark Trap,Blood,band of skulls,SOJXSLN1311AFDB3FE,ARONQXK122BCFC9386,"I want it, to be over\nTurn the boat around an...",en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,0.384,0.787,8,-5.861,0,0.3350,0.142000,0.000028,0.3900,0.5440,169.700,204707,Hiphop,Beautiful Girl,george harrison,SOCJCDA12A81C20704,ARW9QSZ1187FB4B93E,Never seen such a beautiful girl\nGot me shaki...,en
1123,0.668,0.565,6,-5.879,1,0.5050,0.171000,0.000000,0.1240,0.1420,140.489,261867,Hiphop,Too Much,spice girls,SOXFJWX1373299A525,AR7VWZ11187B98DA42,"Love is blind, as far as the eye can see,\ndee...",en
1124,0.653,0.869,9,-5.050,0,0.0638,0.154000,0.000000,0.1170,0.3700,178.032,210013,Hiphop,The Rain,akon,SOCSUOL12AAA8C6707,ARCGJ6U1187FB4D01F,Yeah\nOh diddy oh diddy oh\nAhhhh\nAhhhh\nKonv...,en
1125,0.810,0.546,8,-9.019,0,0.0686,0.013700,0.000000,0.0830,0.6150,191.812,263960,Hiphop,Low Down,black label society,SOXFONK1311AFE035C,ARQFUGM1187FB3E24E,You leave me hanging high\nLost my sense to co...,en


In [24]:
data.drop_duplicates(subset=['songID'], keep='first', inplace=False, ignore_index=False)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre,song_name,artistname,songID,artistID,Lyric,language
0,0.849,0.648,7,-6.188,1,0.0832,0.072500,0.005920,0.0984,0.1960,212.150,111020,Dark Trap,Paris,kate nash,SOZBGPI12AB018A2DA,ARUZIPI1187FB5454A,"You've come so far, well done darling\nWe knew...",en
4,0.836,0.706,1,-4.065,1,0.0497,0.000052,0.000000,0.0927,0.0374,120.045,76042,Dark Trap,Blood,band of skulls,SOJXSLN1311AFDB3FE,ARONQXK122BCFC9386,"I want it, to be over\nTurn the boat around an...",en
8,0.849,0.522,1,-5.514,1,0.1510,0.001900,0.000000,0.2090,0.0582,119.975,138266,Dark Trap,Tears,kylie minogue,SOPQHSW12A8AE47CE4,AR8V1X71187B9B919B,Never would say\nI never needed anyone\nNot no...,en
11,0.849,0.522,1,-5.514,1,0.1510,0.001900,0.000000,0.2090,0.0582,119.975,138266,Dark Trap,Tears,teenage fanclub,SOLSKMB1315CD463B4,ARHPPJK1187B9AC897,There's no future so don't fake it\nDon't know...,en
14,0.475,0.788,10,-4.851,0,0.3420,0.231000,0.000000,0.1280,0.2600,206.056,237101,Dark Trap,Crazy,seal,SODHNVL12A58A7B347,ARAIABB1187B9AC6E2,"In a church, by the face,\nHe talks about the ...",en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,0.384,0.787,8,-5.861,0,0.3350,0.142000,0.000028,0.3900,0.5440,169.700,204707,Hiphop,Beautiful Girl,george harrison,SOCJCDA12A81C20704,ARW9QSZ1187FB4B93E,Never seen such a beautiful girl\nGot me shaki...,en
1123,0.668,0.565,6,-5.879,1,0.5050,0.171000,0.000000,0.1240,0.1420,140.489,261867,Hiphop,Too Much,spice girls,SOXFJWX1373299A525,AR7VWZ11187B98DA42,"Love is blind, as far as the eye can see,\ndee...",en
1124,0.653,0.869,9,-5.050,0,0.0638,0.154000,0.000000,0.1170,0.3700,178.032,210013,Hiphop,The Rain,akon,SOCSUOL12AAA8C6707,ARCGJ6U1187FB4D01F,Yeah\nOh diddy oh diddy oh\nAhhhh\nAhhhh\nKonv...,en
1125,0.810,0.546,8,-9.019,0,0.0686,0.013700,0.000000,0.0830,0.6150,191.812,263960,Hiphop,Low Down,black label society,SOXFONK1311AFE035C,ARQFUGM1187FB3E24E,You leave me hanging high\nLost my sense to co...,en


In [25]:
#making a new dataset with temporary data
comp_data = data.select_dtypes(include=['float64','int64'])
comp_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1127 entries, 0 to 1126
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1127 non-null   float64
 1   energy            1127 non-null   float64
 2   key               1127 non-null   int64  
 3   loudness          1127 non-null   float64
 4   mode              1127 non-null   int64  
 5   speechiness       1127 non-null   float64
 6   acousticness      1127 non-null   float64
 7   instrumentalness  1127 non-null   float64
 8   liveness          1127 non-null   float64
 9   valence           1127 non-null   float64
 10  tempo             1127 non-null   float64
 11  duration_ms       1127 non-null   int64  
dtypes: float64(9), int64(3)
memory usage: 114.5 KB
